In [1]:
from kedmi.models.generator import *
from kedmi.models.discri import *
from kedmi.utils.helper import *
import torch
import os
import numpy as np
from kedmi.utils.kedmi_attack import mnist_inversion, dist_inversion

### load specified configuration and specify environment variables

In [2]:
CONFIG_PATH = "config/kedmi/config/celeba/attacking/celeba_cust.json"
configuration = load_json(json_file=CONFIG_PATH)

save_folder = os.path.join(
    f"{configuration['dataset']['name']}_{configuration['dataset']['model_name']}",
    configuration["attack"]["variant"],
)
prefix = os.path.join(
    os.path.join(configuration["root_path"], "kedmi_300ids"), save_folder
)
save_dir = os.path.join(prefix, "latent")
save_img_dir = os.path.join(
    prefix, "imgs_{}".format(configuration["attack"]["variant"])
)

os.makedirs(prefix, exist_ok=True)
os.makedirs(save_img_dir, exist_ok=True)
os.makedirs(save_dir, exist_ok=True)
os.makedirs(configuration["dataset"]["p_reg_path"], exist_ok=True)

### load trained models

In [3]:
targetnets, E, G, D, n_classes, fea_mean, fea_logvar = get_attack_model(configuration)

['VGG16', 'VGG16']
path_G checkpoints/kedmi/GAN/celeba/VGG16/improved_celeba_G.tar
path_D checkpoints/kedmi/GAN/celeba/VGG16/improved_celeba_D.tar


/home/bot/coding/bachelorarbeit/ba_code/kedmi/models/discri.py:79: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.
  init.normal(self.T, 0, 1)


0
VGG16
Load classifier VGG16 at checkpoints/kedmi/target_ckp/VGG16.tar


/home/bot/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/bot/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_BN_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_BN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


1
VGG16
Load classifier VGG16 at checkpoints/kedmi/target_ckp/VGG16.tar


### set necessary params

In [4]:
N = 5
bs = 60

### Run attack

In [5]:
# Begin attacking
for i in range(1):
    iden = torch.from_numpy(np.arange(bs))

    # evaluate on the first 300 identities only
    target_cosines = 0
    eval_cosines = 0
    for idx in range(5):
        iden = iden % n_classes
        print(
            "--------------------- Attack batch [%s]------------------------------"
            % idx
        )
        print("Iden:{}".format(iden))
        save_dir_z = "{}/{}_{}".format(save_dir, i, idx)
        print("kedmi")

        dist_inversion(
            G,
            D,
            targetnets,
            E,
            iden,
            lr=configuration["attack"]["lr"],
            iter_times=configuration["attack"]["iters_mi"],
            momentum=0.9,
            lamda=100,
            clip_range=1,
            improved=configuration['attack']['improved_flag'],
            num_seeds=configuration['attack']['num_seeds'],
            used_loss=configuration['attack']['loss'],
            prefix=save_dir_z,
            save_img_dir=os.path.join(save_img_dir, "{}_".format(idx)),
            fea_mean=fea_mean,
            fea_logvar=fea_logvar,
            lam=configuration["attack"]["lam"],
            clipz=configuration['attack']['clipz'],
        )
        iden = iden + bs

--------------------- Attack batch [0]------------------------------
Iden:tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
        36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53,
        54, 55, 56, 57, 58, 59])
kedmi
criterion:logit_loss
tensor([[-14.5335, -15.7290, -13.0612,  ..., -16.5516, -15.0461, -14.8810],
        [-16.3560, -21.6245, -20.8313,  ..., -16.6473, -15.1798, -18.8965],
        [-15.7243, -16.7756, -12.4205,  ..., -12.9625, -12.0908, -16.9630],
        ...,
        [-16.4053, -16.7750, -20.2339,  ..., -15.8415, -14.6272, -15.8185],
        [-12.4188, -17.9976, -16.7324,  ..., -17.6314, -19.1476, -18.1799],
        [-11.0815, -17.2083, -21.5655,  ..., -15.8791, -17.7278, -15.2674]],
       device='cuda:0', grad_fn=<AddmmBackward0>)
torch.Size([60, 1000])
torch.Size([60, 1000])
tensor([[-15.5337, -13.7604, -15.0633,  ..., -19.0237, -18.3

/home/bot/.local/lib/python3.10/site-packages/torch/nn/parallel/comm.py:227: UserWarning: Using -1 to represent CPU tensor is deprecated. Please use a device object or string instead, e.g., "cpu".
  warnings.warn(


torch.Size([60, 1000])
torch.Size([60, 1000])
tensor([[-12.5372, -18.8626, -16.3484,  ..., -16.7543, -18.7840, -18.7207],
        [-15.2312, -13.2150, -18.5398,  ..., -15.9043, -13.3358, -14.8370],
        [-16.2851, -20.4362, -20.2565,  ..., -13.1798, -17.8322, -15.9593],
        ...,
        [-17.5826, -17.1679, -18.2639,  ..., -22.3893, -19.0860, -18.6612],
        [-16.6056, -16.5862, -18.4062,  ..., -14.1063, -12.4852, -11.6360],
        [-17.3960, -16.8748, -17.8101,  ..., -15.6079, -15.2336, -14.2277]],
       device='cuda:0', grad_fn=<AddmmBackward0>)
torch.Size([60, 1000])
torch.Size([60, 1000])
tensor([[-16.0494, -16.8391, -19.5120,  ..., -18.9522, -15.3610, -16.4739],
        [ -9.6460, -11.1410, -14.0723,  ..., -15.6311, -14.0035, -12.4323],
        [-19.5309, -14.1626, -18.0175,  ..., -20.1880, -19.3038, -16.9919],
        ...,
        [-13.5220, -18.0244, -19.1770,  ..., -15.7211, -14.4183, -14.7680],
        [-15.3647, -14.4510, -15.1202,  ..., -14.7086, -10.9742, -12.90

KeyboardInterrupt: 